In [1]:

import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')


## 각 칼럼 확인

In [3]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
train.groupby('Transported')['Age'].mean()

Transported
False    29.922858
True     27.748834
Name: Age, dtype: float64

In [7]:
train.groupby('Transported')['VIP'].mean()

Transported
False    0.029175
True     0.017782
Name: VIP, dtype: float64

In [8]:
train.groupby(['Transported','VIP']).size()

Transported  VIP  
False        False    4093
             True      123
True         False    4198
             True       76
dtype: int64

In [9]:
train.groupby(['VIP','HomePlanet']).size()

VIP    HomePlanet
False  Earth         4487
       Europa        1958
       Mars          1653
True   Europa         131
       Mars            63
dtype: int64

In [10]:
train.groupby(['Transported','HomePlanet']).size()

Transported  HomePlanet
False        Earth         2651
             Europa         727
             Mars           839
True         Earth         1951
             Europa        1404
             Mars           920
dtype: int64

## HomePlanet Null 처리

HomePlanet Null 채우기 
- 1. PassengerId 를 group으로 만들어서 같은 그룹에 같은 HomePlanet을 준다. (ex: 0001 : Earth, 0002 : Europa)
- 2. PassengerId 를 group화 해서 null값을 채워도 111개가 남음 그래서 Name기준으로 다시 HomePlanet 값을 채운다
- 3. Destination 을 기준으로 HomePlanet을 채운다

In [11]:
# null 값
'''PassengerId    0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0'''

'PassengerId    0\nHomePlanet      201\nCryoSleep       217\nCabin           199\nDestination     182\nAge             179\nVIP             203\nRoomService     181\nFoodCourt       183\nShoppingMall    208\nSpa             183\nVRDeck          188\nName            200\nTransported       0'

In [12]:
train['HomePlanet'].value_counts()

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

In [13]:
train['Group'] = train['PassengerId'].str[:4]
# train['PassengerId'].split('_')str[0]

In [14]:
train['PassengerId'].str[:4]

0       0001
1       0002
2       0003
3       0003
4       0004
        ... 
8688    9276
8689    9278
8690    9279
8691    9280
8692    9280
Name: PassengerId, Length: 8693, dtype: object

In [15]:
train.groupby(['Group','HomePlanet']).size()

Group  HomePlanet
0001   Europa        1
0002   Earth         1
0003   Europa        2
0004   Earth         1
0005   Earth         1
                    ..
9275   Europa        3
9276   Europa        1
9278   Earth         1
9279   Earth         1
9280   Europa        2
Length: 6107, dtype: int64

In [16]:
home_group = train.groupby(['Group','HomePlanet']).size().reset_index()

In [17]:
home_group

,Group,HomePlanet,0
0,0001,Europa,1
1,0002,Earth,1
2,0003,Europa,2
3,0004,Earth,1
4,0005,Earth,1
...,...,...,...
6102,9275,Europa,3
6103,9276,Europa,1
6104,9278,Earth,1
6105,9279,Earth,1


In [18]:
home_group[home_group.duplicated()]
# group이 중복된 것이 없다 각 그룹별로 행성 데이터가 존재 한다 (Earth, Europa, Mars)

,Group,HomePlanet,0


In [19]:
train[train['HomePlanet'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group
59,0064_02,NaN,True,E/3/S,TRAPPIST-1e,33.0,False,0.0,0.0,NaN,0.0,0.0,Colatz Keen,True,0064
113,0119_01,NaN,False,A/0/P,TRAPPIST-1e,39.0,False,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,False,0119
186,0210_01,NaN,True,D/6/P,55 Cancri e,24.0,False,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,True,0210
225,0242_01,NaN,False,F/46/S,TRAPPIST-1e,18.0,False,313.0,1.0,691.0,283.0,0.0,Almone Sté,False,0242
234,0251_01,NaN,True,C/11/S,55 Cancri e,54.0,False,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,True,0251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,NaN,False,E/582/P,TRAPPIST-1e,25.0,False,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,False,9084
8613,9194_01,NaN,False,E/603/S,55 Cancri e,53.0,False,0.0,4017.0,0.0,13.0,3147.0,NaN,False,9194
8666,9248_01,NaN,False,F/1792/S,55 Cancri e,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,True,9248
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False,9257


In [20]:
home_dict={}
for g, hp in zip(home_group['Group'], home_group['HomePlanet']):
    home_dict[g]=hp

In [21]:
train['HomePlanet'].fillna(train['Group'].map(home_dict), inplace=True)
# train['Group'] 이 0 이면 Europa, 1이면 Earth 맵핑

In [22]:
train['HomePlanet'].isnull().sum()
# null 값이 111개 존재 한다 이 값은 다른 방법으로 처리 해야 함

111

In [23]:
train['Name2'] = train['Name'].str.split(' ').str[1]

In [24]:
name_group = train.groupby(['Name2','HomePlanet']).size().reset_index()

In [25]:
name_dict={}
for n, hp in zip(name_group['Name2'], name_group['HomePlanet']):
    name_dict[n]=hp

In [26]:
train['HomePlanet'].fillna(train['Name2'].map(name_dict), inplace=True)
# train['Group'] 이 0 이면 Europa, 1이면 Earth 맵핑

In [27]:
train[train['HomePlanet'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,Name2
225,0242_01,NaN,False,F/46/S,TRAPPIST-1e,18.0,False,313.0,1.0,691.0,283.0,0.0,Almone Sté,False,0242,Sté
234,0251_01,NaN,True,C/11/S,55 Cancri e,54.0,False,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,True,0251,Amsive
807,0853_01,NaN,True,A/9/S,55 Cancri e,38.0,False,0.0,0.0,0.0,0.0,0.0,Hamelik Ageurante,True,0853,Ageurante
1855,1978_01,NaN,True,G/311/S,TRAPPIST-1e,19.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True,1978,NaN
2274,2443_01,NaN,False,D/72/P,TRAPPIST-1e,31.0,False,1458.0,421.0,76.0,0.0,0.0,NaN,False,2443,NaN
2631,2817_01,NaN,False,F/584/P,TRAPPIST-1e,25.0,False,237.0,0.0,910.0,0.0,12.0,Sealfs Sutty,False,2817,Sutty
3091,3331_01,NaN,False,F/631/S,TRAPPIST-1e,40.0,False,666.0,4.0,83.0,0.0,50.0,NaN,True,3331,NaN
4548,4840_01,NaN,True,F/915/S,TRAPPIST-1e,36.0,False,0.0,0.0,NaN,0.0,0.0,NaN,True,4840,NaN
5252,5603_01,NaN,False,E/365/S,TRAPPIST-1e,34.0,False,170.0,1256.0,0.0,3926.0,7121.0,Kocha Cluitty,False,5603,Cluitty
5634,5989_01,NaN,False,F/1141/S,TRAPPIST-1e,20.0,False,0.0,0.0,NaN,703.0,0.0,Darrie Holcompton,False,5989,Holcompton


In [28]:
train.groupby(['Destination','HomePlanet']).size()

Destination    HomePlanet
55 Cancri e    Earth          700
               Europa         902
               Mars           196
PSO J318.5-22  Earth          725
               Europa          19
               Mars            51
TRAPPIST-1e    Earth         3177
               Europa        1214
               Mars          1515
dtype: int64

In [29]:
train['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [30]:
des_dict={'55 Cancri e':'Europa','PSO J318.5-22':'Earth', 'TRAPPIST-1e':'Earth'}

In [31]:
des_dict

{'55 Cancri e': 'Europa', 'PSO J318.5-22': 'Earth', 'TRAPPIST-1e': 'Earth'}

In [32]:
train['HomePlanet'].fillna(train['Destination'].map(des_dict), inplace=True)
# train['HomePlanet'].fillna(train['Destination'].map({'55 Cancri e':'Europa',
# 'PSO J318.5-22':'Earth', 'TRAPPIST-1e':'Earth'}), inplace=True)

In [33]:
train['HomePlanet'].isnull().sum()

0

In [34]:
train['VIP'].value_counts(dropna=False)
# VIP는 별 관련이 없어서 False로 채움

False    8291
NaN       203
True      199
Name: VIP, dtype: int64

In [35]:
train[train['VIP']==True]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,Name2
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,Susent
108,0112_01,Europa,False,B/1/S,55 Cancri e,48.0,True,0.0,2537.0,87.0,17.0,13.0,Moth Cowtale,True,0112,Cowtale
120,0128_01,Mars,False,D/3/S,TRAPPIST-1e,61.0,True,2353.0,334.0,9.0,316.0,2.0,Grohs Fles,False,0128,Fles
214,0224_01,Mars,False,F/42/S,TRAPPIST-1e,32.0,True,181.0,0.0,5.0,1634.0,0.0,Blues Queen,False,0224,Queen
291,0321_01,Mars,False,F/61/S,TRAPPIST-1e,59.0,True,1018.0,0.0,209.0,0.0,0.0,Quites Bache,False,0321,Bache
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8579,9158_01,Europa,True,B/298/P,55 Cancri e,30.0,True,0.0,0.0,0.0,0.0,0.0,Magnon Maglible,True,9158,Maglible
8614,9194_02,Europa,False,E/603/S,TRAPPIST-1e,32.0,True,1003.0,909.0,0.0,0.0,15.0,Tachba Subwor,False,9194,Subwor
8621,9197_02,Europa,False,C/308/P,NaN,41.0,True,0.0,7964.0,0.0,3238.0,5839.0,Aludram Platch,False,9197,Platch
8652,9230_01,Europa,False,C/342/S,TRAPPIST-1e,36.0,True,0.0,5600.0,715.0,2868.0,971.0,NaN,True,9230,NaN


In [36]:
train['VIP'].fillna(False, inplace=True)

In [37]:
train['VIP'].isnull().sum()

0

In [38]:
train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP               0
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
Group             0
Name2           200
dtype: int64

In [39]:
fee_list = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

In [40]:
train['RoomService'].median()

0.0

In [41]:
fee_list

['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [42]:
train['RoomService'].median()

0.0

In [43]:
train['FoodCourt'].median()

0.0

In [44]:
for col in fee_list:
    train[col].fillna(train[col].median(),inplace=True)

In [45]:
train['Fee'] = train['RoomService']+train['FoodCourt']+train['ShoppingMall']+train['Spa']+train['VRDeck']         

In [46]:
train['Fee'].isnull().sum()

0

In [47]:
train['deck'] = train['Cabin'].str.split('/').str[0]

In [48]:
train['side'] = train['Cabin'].str.split('/').str[-1]

In [49]:
train['Cabin']

0          B/0/P
1          F/0/S
2          A/0/S
3          A/0/S
4          F/1/S
          ...   
8688      A/98/P
8689    G/1499/S
8690    G/1500/S
8691     E/608/S
8692     E/608/S
Name: Cabin, Length: 8693, dtype: object

In [50]:
train.groupby(['HomePlanet','deck']).size()

HomePlanet  deck
Earth       D          1
            E        404
            F       1652
            G       2559
Europa      A        256
            B        779
            C        747
            D        192
            E        133
            T          5
Mars        D        285
            E        339
            F       1142
dtype: int64

Earth : G , Europa : B

In [51]:
train['deck'].fillna(train['HomePlanet'].map({'Earth':'G','Europa':'B', 'Mars':'F'}), inplace=True)

In [52]:
train['deck'].isnull().sum()

0

In [53]:
side_group = train.groupby(['Group','side']).size().reset_index()

In [54]:
side_group[side_group['Group'].duplicated()]

,Group,side,0


In [55]:
side_dict={}
for s,g in zip(side_group['Group'], side_group['side']):
    side_dict[s]=g

In [56]:
side_dict

{'0001': 'P',
 '0002': 'S',
 '0003': 'S',
 '0004': 'S',
 '0005': 'P',
 '0006': 'S',
 '0007': 'S',
 '0008': 'P',
 '0009': 'P',
 '0010': 'S',
 '0011': 'P',
 '0014': 'P',
 '0015': 'P',
 '0016': 'P',
 '0017': 'P',
 '0020': 'S',
 '0022': 'P',
 '0024': 'S',
 '0025': 'S',
 '0026': 'P',
 '0028': 'P',
 '0030': 'S',
 '0031': 'P',
 '0034': 'S',
 '0035': 'P',
 '0036': 'S',
 '0038': 'S',
 '0039': 'P',
 '0041': 'P',
 '0043': 'P',
 '0044': 'P',
 '0045': 'P',
 '0050': 'S',
 '0051': 'S',
 '0052': 'S',
 '0053': 'S',
 '0056': 'S',
 '0058': 'S',
 '0061': 'S',
 '0062': 'S',
 '0064': 'S',
 '0066': 'P',
 '0067': 'S',
 '0068': 'S',
 '0069': 'S',
 '0070': 'P',
 '0071': 'P',
 '0072': 'S',
 '0073': 'P',
 '0074': 'S',
 '0076': 'S',
 '0077': 'P',
 '0078': 'S',
 '0081': 'S',
 '0082': 'P',
 '0084': 'S',
 '0085': 'S',
 '0086': 'P',
 '0088': 'S',
 '0090': 'S',
 '0091': 'S',
 '0092': 'P',
 '0097': 'S',
 '0098': 'P',
 '0099': 'P',
 '0102': 'S',
 '0103': 'S',
 '0105': 'P',
 '0107': 'S',
 '0108': 'S',
 '0110': 'P',
 '0111

In [57]:
train['deck'].isnull().sum()

0

In [58]:
train['side'].fillna(train['Group'].map(side_dict),inplace=True)

In [59]:
train['side'].isnull().sum()

99

In [60]:
train['side'].value_counts()

S    4343
P    4251
Name: side, dtype: int64

In [61]:
train['side'].fillna('S',inplace=True)

In [62]:
dest_group = train.groupby(['Group','Destination']).size().reset_index()

In [63]:
dest_group[dest_group['Group'].duplicated()]

,Group,Destination,0
8,0008,TRAPPIST-1e,1
17,0017,TRAPPIST-1e,1
19,0020,PSO J318.5-22,1
20,0020,TRAPPIST-1e,3
36,0044,PSO J318.5-22,1
...,...,...,...
6836,9219,TRAPPIST-1e,1
6838,9220,TRAPPIST-1e,2
6844,9227,TRAPPIST-1e,3
6847,9231,TRAPPIST-1e,2


In [64]:
dest_group = dest_group.sort_values(0, ascending=False)

In [65]:
dest_group = dest_group.drop_duplicates()

In [66]:
dest_dict={}
for g , d in zip(dest_group['Group'], dest_group['Destination']):
    dest_dict[g]=d

In [67]:
train['Destination'].fillna(train['Group'].map(dest_dict),inplace=True)

In [68]:
train['Destination'].isnull().sum()

103

In [69]:
train.groupby(['HomePlanet','Destination']).size()

HomePlanet  Destination  
Earth       55 Cancri e       704
            PSO J318.5-22     736
            TRAPPIST-1e      3203
Europa      55 Cancri e       915
            PSO J318.5-22      19
            TRAPPIST-1e      1224
Mars        55 Cancri e       201
            PSO J318.5-22      51
            TRAPPIST-1e      1537
dtype: int64

In [70]:
train['Destination'].value_counts()

TRAPPIST-1e      5964
55 Cancri e      1820
PSO J318.5-22     806
Name: Destination, dtype: int64

In [71]:
train['Destination'].fillna('TRAPPIST-1e',inplace=True)

In [72]:
train['Destination'].isnull().sum()

0

In [73]:
train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Destination       0
Age             179
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Group             0
Name2           200
Fee               0
deck              0
side              0
dtype: int64

In [74]:
# RoomService, FoodCourt, ShoppingMall,Spa,VRDeck
train['CryoSleep']

0       False
1       False
2       False
3       False
4       False
        ...  
8688    False
8689     True
8690    False
8691    False
8692    False
Name: CryoSleep, Length: 8693, dtype: object

In [75]:
train[train['CryoSleep'].isnull()]['Fee']

92         0.0
98       703.0
104     2018.0
111        0.0
152      990.0
         ...  
8620       0.0
8651       0.0
8664       0.0
8675    2056.0
8687    3540.0
Name: Fee, Length: 217, dtype: float64

In [76]:
train['CryoSleep'] = np.where(train['CryoSleep'] >0 ,False, True)

In [77]:
train['CryoSleep'].isnull().sum()

0

In [78]:
train['Transported'].value_counts()

True     4378
False    4315
Name: Transported, dtype: int64

In [79]:
train['Age'].fillna(train['Age'].median(),inplace=True)

In [80]:
del_col = ['PassengerId', 'Name', 'Name2','Cabin' ]

In [81]:
train = train.drop(del_col, axis=1)

In [82]:
train.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Group           0
Fee             0
deck            0
side            0
dtype: int64

In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   object 
 1   CryoSleep     8693 non-null   bool   
 2   Destination   8693 non-null   object 
 3   Age           8693 non-null   float64
 4   VIP           8693 non-null   bool   
 5   RoomService   8693 non-null   float64
 6   FoodCourt     8693 non-null   float64
 7   ShoppingMall  8693 non-null   float64
 8   Spa           8693 non-null   float64
 9   VRDeck        8693 non-null   float64
 10  Transported   8693 non-null   bool   
 11  Group         8693 non-null   object 
 12  Fee           8693 non-null   float64
 13  deck          8693 non-null   object 
 14  side          8693 non-null   object 
dtypes: bool(3), float64(7), object(5)
memory usage: 840.6+ KB


In [131]:
print(train['HomePlanet'].unique())
print(train['Destination'].unique())
print(train['Group'].unique())
print(train['deck'].unique())
print(train['side'].unique())

[1 0 2]


In [90]:
from sklearn.preprocessing import LabelEncoder

In [99]:
obj_col=[]
for col,types in zip(train.dtypes.index, train.dtypes.values):
    if types=='object':
        obj_col.append(col)

In [100]:
obj_col

['HomePlanet', 'Destination', 'Group', 'deck', 'side']

In [124]:
import pickle

In [126]:
for col in obj_col:
    label = LabelEncoder()
    train[col]=label.fit_transform(train[col])
    filename = 'label_'+col+'.pkl'
    with open('rabel_model.pkl', 'wb') as f:
        pickle.dump(label, f)

In [103]:
X = train.drop('Transported',axis=1)
Y = train['Transported']

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

- train 데이터는 문제,정답있고 test는 문제만 있는 것 train을 통해 모델을 만들고 test를 모델에 돌려서 결과제출

In [116]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=10)

In [118]:
rfc_model = RandomForestClassifier()

In [120]:
rfc_model.fit(x_train,y_train)

RandomForestClassifier()

In [122]:
rfc_model.score(x_test,y_test)

0.7947096032202415